# Calculating a security's risk in Python

In [55]:
# Import the necessary libraries for the project

import numpy as np
import pandas as pd
from pandas_datareader import data as wb
import matplotlib.pyplot as plt


In [56]:
# Create a list of stocks and name it as "tickers"

tickers = ['KO', 'TSLA', 'AAPL', 'MSFT']

# Create a dataframe and name it as "sec_data"

sec_data = pd.DataFrame()

for t in tickers:
    sec_data[t] = wb.DataReader(t, data_source = 'yahoo', start = '2015-1-1')['Adj Close']
    

In [57]:
sec_data.head()

,KO,TSLA,AAPL,MSFT
Date,,,,
2015-01-02,35.171711,219.309998,99.766006,41.647892
2015-01-05,35.171711,210.089996,96.955429,41.264908
2015-01-06,35.438805,211.279999,96.964584,40.659245
2015-01-07,35.881165,210.949997,98.324242,41.175831
2015-01-08,36.315170,210.619995,102.102089,42.387146


In [58]:
# Cacculate daily log returns for all stocks

sec_returns = np.log(sec_data / sec_data.shift(1))
sec_returns

,KO,TSLA,AAPL,MSFT
Date,,,,
2015-01-02,NaN,NaN,NaN,NaN
2015-01-05,0.000000,-0.042950,-0.028576,-0.009238
2015-01-06,0.007565,0.005648,0.000094,-0.014786
2015-01-07,0.012405,-0.001563,0.013925,0.012625
2015-01-08,0.012023,-0.001566,0.037703,0.028994
...,...,...,...,...
2020-08-24,0.014488,-0.017608,0.011889,0.003140
2020-08-25,-0.001252,0.004528,-0.008238,0.012926
2020-08-26,0.005205,0.062192,0.013507,0.021389


In [59]:
# Calculate the average daily return for all stocks in "tickers" list:

sec_returns[tickers].mean()

KO      0.000245
TSLA    0.001623
AAPL    0.001131
MSFT    0.001197
dtype: float64

In [60]:
# Calculate the annual return for all stocks in "tickers" list:

sec_returns[tickers].mean() * 250

KO      0.061161
TSLA    0.405864
AAPL    0.282697
MSFT    0.299171
dtype: float64

In [61]:
# Calculate the daily standard deviation for all stocks in "tickers" list:

sec_returns[tickers].std()

KO      0.012013
TSLA    0.033452
AAPL    0.018195
MSFT    0.017408
dtype: float64

In [62]:
# Calculate the annual standard deviation for all stocks in "tickers" list:

sec_returns[tickers].std() * np.sqrt(250)

KO      0.189939
TSLA    0.528922
AAPL    0.287686
MSFT    0.275248
dtype: float64

# Calculating Covariance and Correlation

In [63]:
sec_returns.head()

,KO,TSLA,AAPL,MSFT
Date,,,,
2015-01-02,NaN,NaN,NaN,NaN
2015-01-05,0.000000,-0.042950,-0.028576,-0.009238
2015-01-06,0.007565,0.005648,0.000094,-0.014786
2015-01-07,0.012405,-0.001563,0.013925,0.012625
2015-01-08,0.012023,-0.001566,0.037703,0.028994


In [64]:
# Calcukate the covaraince of all stocks

covariance_matrix = sec_returns.cov()
covariance_matrix

,KO,TSLA,AAPL,MSFT
KO,0.000144,0.000093,0.000090,0.000099
TSLA,0.000093,0.001119,0.000224,0.000234
AAPL,0.000090,0.000224,0.000331,0.000216
MSFT,0.000099,0.000234,0.000216,0.000303


In [65]:
# Calcukate the correlation of all stocks between each other

correlation_matrix = sec_returns.corr()
correlation_matrix

,KO,TSLA,AAPL,MSFT
KO,1.000000,0.232594,0.410221,0.475034
TSLA,0.232594,1.000000,0.367753,0.401600
AAPL,0.410221,0.367753,1.000000,0.682381
MSFT,0.475034,0.401600,0.682381,1.000000


In [66]:
## Correlation between stock prices and returns are not the same thing!
## Do not annualize the correlation table!

# Calculating Portfolio Risk

In [176]:
# First, we need to assign weights to all stocks in our portfolio
# If we invest same amount of money to each stock:

weights = np.array([0.25, 0.25, 0.25, 0.25])


## Portfolio variance

In [177]:
portfolio_var = np.dot(weights.T, np.dot(sec_returns.cov() * 250, weights)) 

In [178]:
portfolio_var

0.059532178439548125

In [179]:
print(str(round(portfolio_var, 5) * 100) + '%')

5.953%


## Portfolio volatility

In [180]:
portfolio_vol = np.sqrt(np.dot(weights.T, np.dot(sec_returns.cov() * 250, weights)))
portfolio_vol

0.243992168807829

In [181]:
print(str(round(portfolio_vol, 5) * 100) + '%')

24.399%


## Calculating Diversifiable and Non-diversifiable risk of a portfolio

In [182]:
weights

array([0.25, 0.25, 0.25, 0.25])

In [183]:
variance_all_stocks = sec_returns[tickers].var() * 250
variance_all_stocks

KO      0.036077
TSLA    0.279759
AAPL    0.082763
MSFT    0.075761
dtype: float64

In [184]:
## Diversifiable Risk = portfolio variance - weighted annual variances

diversifiable_risk = portfolio_var - ((weights[0] ** 2 * variance_all_stocks[0]) + (weights[1] ** 2 * variance_all_stocks[1]) + (weights[2] ** 2 * variance_all_stocks[2]) + (weights[3] ** 2 * variance_all_stocks[3]))

In [185]:
diversifiable_risk

0.029884657451493483

In [186]:
print(str(round(diversifiable_risk * 100, 3)) + '%')

2.988%


## Non-diversifiable risk

In [187]:
non_div_risk_1 = portfolio_var - diversifiable_risk
non_div_risk_1


0.02964752098805464

In [188]:
non_div_risk_2 = (weights[0] ** 2 * variance_all_stocks[0]) + (weights[1] ** 2 * variance_all_stocks[1]) + (weights[2] ** 2 * variance_all_stocks[2]) + (weights[3] ** 2 * variance_all_stocks[3])
non_div_risk_2

0.02964752098805464

In [189]:
print(str(round(non_div_risk_2, 5) * 100) + '%')

2.965%


In [190]:
non_div_risk_1 == non_div_risk_2

True